# Biopython Intro

## What does Biopython do?

* Biopython is a Python library for handling genomics (and some other omics) tasks in Python.
* Some of these can be useful for writing Jupyter notebooks or other Python reporting
* Some are less useful for us but are used in allied fields - e.g. bacteriology

I'll cover some of the more relevant tasks, but it's good to know about other functionalities, in case you end up needing them for papers.
   
* Most of the documentation is in the tutorial and cookbook: http://biopython.org/DIST/docs/tutorial/Tutorial.html​
* But for SeqIO and some other objects I find it easy to use the wiki, linked here: https://biopython.org/wiki/Documentation 


### Some handy Biopython functionality

* Parsers, converters writers for FASTAs and other data formats​
* Calling Entrez and parse results
* Sequence-level tasks: translate, reverse-transcribe​
* Wrappers for calling local and online BLAST and some common aligners​

### Could be useful for some papers and reports?

* Graphics, including chromosome drawing​

### More niche, but interesting

* Population genetics and phylogenomics​
* Motif detection, kmer-based analyses, and some supervised learning bits and bobs
* Modules for handling crystal structures of biological macromolecules​

## Imports

Biopython can be installed with pip or conda.
It gets imported as 'Bio'.

## Setting up this notebook
1. Make a copy of 'biopython.config.template' in the 'biopython' directory, and rename your copy to 'biopython.config'. This is in '.gitignore' so you should be able to add your config details without it accidentally git adding/committing.
2. Set YOUR_EMAIL in 'biopython.config' to equal your work email address - it will be used to query NCBI.
3. Set PARENT_PATH in 'biopython.config' to equal where your example data is kept.
4. Download NC_000009.12 as a FASTA from NCBI, and save it as "NC_000009.12.fasta" in your PARENT_PATH. 

In [3]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import Entrez
import time
from pathlib import Path
from configparser import ConfigParser

config = ConfigParser(allow_no_value=True)
config.read('./biopython.config')


YOUR_EMAIL = config["config"]["YOUR_EMAIL"]
PARENT_PATH = config["config"]["PARENT_PATH"]

## Parsing

#### FASTA
Biopython lets you easily open and parse a lot of common bioinformatics file types, including FASTAs. For FASTA files, the '>' line gets parsed as 'id', and the line after it is parsed as 'seq'. For example, for an ABL1 sequence downloaded from NCBI:

In [4]:
parent_path = str(PARENT_PATH)
abl1_path = parent_path + "NC_000009.12.fasta"

for record in SeqIO.parse(abl1_path, "fasta"):
    print(record.id)
    print(record.seq)

FileNotFoundError: [Errno 2] No such file or directory: '"/home/corbin/Documents/Code school/Biopython/"NC_000009.12.fasta'

If the file contained multiple FASTA sequences, the loop above would print all their IDs and sequences.

#### FASTQ

Biopython can parse FASTQ files, but make sure to choose the right parser. Note that FastQC has a PHRED encoding guesser which may help you if you aren't sure which PHRED version your file is using:
- 'fastq' or 'fastq-sanger' - Sanger style FASTQ files, encodes PHRED with an ASCII offset of 33. Illumina pipeline 1.8 makes these.
- 'fastq-illumina' - the OLD Illumina style of encoding PHRED, which differs slightly from the Sanger method! You may need to keep an eye on this for old Illumina data.
- 'fastq-solexa' - ASCII offset of 64. We probably won't use these.

#### Sanger ABI files
* If you're feeling old-school and have Sanger data, you can use Biopython to open and manipulate ABI files, too.

## Working with NCBI Entrez

Biopython provides tools to help you get results from NCBI's Entrez, then parse and work with them. 

#### Don't annoy them
* Make sure to provide your email, and keep the number of API queries at 3 queries per second or below, or 10 if you set an API key - NCBI can ban you if you keep spamming. 
* Note that in my code below, I set 'time.sleep(0.4)' to slow my code execution down. 
* Here I'm only running a single example so it isn't as important, but if you're looping through gene names, it might save you upsetting someone.


#### Basic steps
To query a database, you set 'db' to the correct database type (we're using 'gene') and put your search terms in 'term'. The returned XML is parsed by Biopython into an object. Below, I give an example of fetching the human ABL1 gene. The steps are:
* Get IDs with 'esearch'
* Use 'efetch' on those IDs to get the info we care about
* Write information to a file

Note that 'efetch' has a lot of return type, 'retmode', options. 'gb' brings back a Genbank style output which is easier to read for a human, but 'xml' is easier to parse with Python.

In [ ]:
Entrez.email = YOUR_EMAIL
handle = Entrez.esearch(db="gene", term="ABL1[GENE] Homo sapiens[ORGN]")
time.sleep(0.4)
record = Entrez.read(handle)

print("The returned object, without digging into it further")
print(record)

print("\nThe list of IDs found by eSearch:")
print(record["IdList"])

print("\nIf there's only one result - try to get its ID")
if record["Count"] == "1":
    esearch_id = int(record["IdList"][0])
    gb_handle = Entrez.efetch(db="gene", rettype="gb", retmode="txt", id=esearch_id)
    time.sleep(0.4)
    print(gb_handle.read())
    print("\nTry to save the file locally if you'll use it later in your process, so that you aren't stressing NCBI servers")
    filename = parent_path + "ABL1_example.gbk"
    filepath = Path(filename)
    if not filepath.is_file():
        with open(filename, "w") as out_handle:
            out_handle.write(gb_handle.read())
            print("Saved: " + filename)
    else:
        print("Skipped file-saving as " + str(filename) + " already exists")


The returned object, without digging into it further
{'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': ['25'], 'TranslationSet': [{'From': 'Homo sapiens[ORGN]', 'To': '"Homo sapiens"[Organism]'}], 'TranslationStack': [{'Term': 'ABL1[GENE]', 'Field': 'GENE', 'Count': '520', 'Explode': 'N'}, {'Term': '"Homo sapiens"[Organism]', 'Field': 'Organism', 'Count': '325491', 'Explode': 'Y'}, 'AND'], 'QueryTranslation': 'ABL1[GENE] AND "Homo sapiens"[Organism]'}

The list of IDs found by eSearch:
['25']

If there's only one result - try to get its ID

1. ABL1
Official Symbol: ABL1 and Name: ABL proto-oncogene 1, non-receptor tyrosine kinase [Homo sapiens (human)]
Other Aliases: ABL, BCR-ABL, CHDSKM, JTK7, bcr/abl, c-ABL, c-ABL1, p150, v-abl
Other Designations: tyrosine-protein kinase ABL1; ABL protooncogene 1 nonreceptor tyrosine kinase; Abelson tyrosine-protein kinase 1; BCR-ABL1 p190; BCR/ABL e8a2 fusion; BCR/ABL1 e1a2 fusion protein; bcr/c-abl oncogene protein; c-abl oncogene 1, recepto

## Working with sequences

Biopython makes it quite easy to work with sequences. Below, as an example, we generate all possible amino acid sequences for ABL1, and select the largest one.

In [ ]:
for record in SeqIO.parse(abl1_path, "fasta"):
    possible_transcriptions = [record.seq, record.seq.reverse_complement()]
    possible_aa_seqs = (transcript[i:].translate(to_stop=True) for i in range(3) for transcript in possible_transcriptions)
    max_aa = max(possible_aa_seqs, key=len)
    print("\nLongest amino acid sequence:")
    print(max_aa)



Longest amino acid sequence:
SNVNTDLFKKYYRGQIWIQKKPQRIIYRNKMHWTPRSRGGATTGPRGGRAGRYAHAT
